In [1]:
# running in python 3.5
import numpy as np
import numba

In [2]:

@numba.jit(nopython = True)
def get_char_pol(A):
    """
    This uses Newton's identities to get the characteristic polynomial
    This is a straightfoward, implementation of the formula 
    I derived for Newton's Identities, that used two different
    forms of differentiation, and equated terms in the underlying 
    power series of the differentiated, factored polynomial
    """
    assert(A.shape[0] == A.shape[1]) # it must be a square matrix
    n = A.shape[0] 
    charpoly_array = np.zeros(n + 1)
    traces_array = np.zeros(n + 1)
    
    traces_array[0] = n
    working_matrix = A.copy()
    
    traces_array[1] = np.trace(working_matrix)
    tracer_idx = 1
    # note trace (A^1) will be in the position 1...    
    # a_0, a_1, a_2, ..., a_n
    charpoly_array[n] = 1
    
    for r in range(1, n):
        # through n-1, then separate handling at the end for a_0
        a_idx = n - r
        running_sum = 0        
        
        working_matrix = working_matrix @ A
        tracer_idx += 1
        traces_array[tracer_idx] = np.trace(working_matrix)
        for j in range(1, r + 1):
            running_sum += charpoly_array[a_idx + j] * traces_array[j]
            # this itself can go in a memo I think... simplisitically with np.zeros and such... 
            # slightly more complicated with a buffer / pivot to which we don't go past
        charpoly_array[a_idx] += - running_sum / r
    
    another_running_sum = 0
    for i in range(1, n + 1):
        another_running_sum += charpoly_array[i] * traces_array[i]
    charpoly_array[0] = -1/n * another_running_sum 
    return charpoly_array


In [3]:
@numba.jit(nopython = True)
def textbook_get_char_pol(A):
    """
    This uses Newton's identities to get the characteristic polynomial
    specifically, this is called:   

    Leverrier–Souriau–Frame Algorithm, and it is problem 7.1.24 
    in Meyer's ""Matrix Analysis and Applied Linear Algebra" 
    """
    assert(A.shape[0] == A.shape[1]) # it must be a square matrix
    n = A.shape[0] 
    charpoly_array = np.zeros(n + 1)
    
    traces_array = np.zeros(n + 1)
    charpoly_array[0] = 1
    # note the ordering in this algorithms is in reverse from the above, 
    # though a reversed array is returned at the end
    
    B = np.identity(n)
    I = np.identity(n)
        
    for k in range(1, n + 1):
        AB_k_minus_one = A @ B
        
        the_scalar = np.trace(AB_k_minus_one) / k
        charpoly_array[k] = - the_scalar    

#         B = -(np.trace(AB_k_minus_one) / k) * I + A @ B
        # either use the above line, or the below 3 lines               
        B = AB_k_minus_one
        for i in range(n):
            B[i,i] += -the_scalar       
    
    # the below returns the reverse of the characteristic polynomial array, so as to conform with ordering:
    # a_0, a_1, a_2, ..., a_n
    return charpoly_array[::-1]
        

In [4]:
m = 10
A = np.random.rand(m*m)
# A = np.random.randint(-5, 10,m*m)

A = np.float64(np.reshape(A, (m,m))) * 10
# numba gets upset at typing issues if A is not in floats...

result = get_char_pol(A)
print(result,"\n\n")

textbook_result = textbook_get_char_pol(A)
print(textbook_result)

# in general the idea is elegant, though there seems to be floating point stability issues, with say, n = 20

[  1.30730011e+08   1.40619079e+07   9.55025379e+06  -8.49162185e+06
   4.21743991e+05  -2.21878738e+05   2.42152638e+04   3.01924722e+03
   2.52416543e+02  -5.72176247e+01   1.00000000e+00] 


[  1.30730011e+08   1.40619079e+07   9.55025379e+06  -8.49162185e+06
   4.21743991e+05  -2.21878738e+05   2.42152638e+04   3.01924722e+03
   2.52416543e+02  -5.72176247e+01   1.00000000e+00]


In [5]:
eigvals = np.linalg.eigvals(A)
polynomials = np.poly(eigvals)
# c[0] * x**(N) + c[1] * x**(N-1) + ... + c[N-1] * x + c[N]
# note that these show up in reversed order...
reversed_polynomial_array = polynomials[::-1]
print(reversed_polynomial_array)


[  1.30730011e+08   1.40619079e+07   9.55025379e+06  -8.49162185e+06
   4.21743991e+05  -2.21878738e+05   2.42152638e+04   3.01924722e+03
   2.52416543e+02  -5.72176247e+01   1.00000000e+00]
